## Setting up local environment for SageMaker 

There are a number of benefits of doing your initial development locally (on your laptop or desktop machine), specifically: 
- You don’t incur any running costs for doing your development locally.
- You can choose preferred IDE which result in more efficient development cycles. 

However, local development runtime also has certain limitations. For instance, you cannot test and profile your code on different hardware devices. Getting the latest GPU devices designed for DL workloads can be impractical and not cost efficient. That’s why, in many cases, you will do initial development and testing of your DL code using a CPU device to troubleshoot initial issues and then do the final code profiling and tweaking on cloud instances with access to target GPU devices. 

SageMaker provides a number of SDKs to allow integration between local environment and AWS cloud. Let’s do a practical example of how to configure your local environment to work with remote SageMaker resources.

### Configuring Python Environment 

We start our configuration by setting up and configuring Python environment with AWS integration. It’s recommended to use Conda environment management software to isolate your SageMaker local environment. You can start by installing Conda on your local machine using appropriate installation method (depends on your local OS). Once Conda is installed, you can create a new Python environment by running the following command in your terminal window: 

```bash
conda create -n sagemaker python=3.9
```

Note that we are explicitly specifying which version of Python interpreter to use in this environment.  Next, we switch to create environment and install AWS and SageMaker SDKs: 

```bash
conda activate sagemaker 
pip install boto3 awscli sagemaker 
```

Let’s review SDKs we just installed: 
- `awscli` is an AWS CLI toolkit which allows you to programmatically work with any AWS service. It also provides mechanism to store and use AWS credentials locally. 
- `boto3` is a Python SDK to manage your AWS resources. It uses credentials established by AWS CLI toolkit to cryptographically sign any management requests and, thus, authenticate in AWS.  
- `sagemaker` Python SDK should be already familiar for you at this point of book as we used it in previous chapters to interact with SageMaker resources such as training jobs or inference endpoints. Unlike `boto3`, SageMaker SDK abstracts many aspects of the management of underlying resources and is generally recommended whenever you need to programmatically manage your SageMaker workloads. 

Before we proceed, we first need to configure AWS credentials. To do so, you will need to run the following command in your terminal and provide your AWS Access and Secret keys: 

```bash
aws configure 
```

You can read the details of how to setup AWS credentials here: https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html 

### Configuring Jupyter environment  

Once we have basic Python environments configured and have your AWS credentials established, we are ready to start Jupyter server. In this example, we will use JupyterLab environment. However, you are free to configure your own IDE for this purpose as many IDEs, such as PyCharm and Visual Studio Code, support Jupyter Notebooks via plugins or natively. The additional benefit of such approach is that you can easily switch between your notebooks and training and inference scripts within the same IDE. 

To install JupyterLab and create a kernel, run the following commands in your terminal: 

```bash
conda install -c conda-forge jupyterlab 
python -m ipykernel install --user --name sagemaker 
```

Next, we start the JupyterLab server on your machine: 

```bash
jupyter lab 
```

Your JupyterLab server should be now available on http://localhost:8888 

## Testing Local Environment 

Once we installed Conda environment and JupyterLab, we can test that we can programmatically interact with SageMaker from local notebook. Please make sure to select to select appropriate `sagemaker` kernel before running cells below.

Let's start by establishing SageMaker session and specifying SageMaker execution role ARN. Please note, that you will need to manually define your execution role. For this, you can navigate to AWS IAM service and look for SageMaker execution role on "Roles" tab (see screenshot below). 

![title](static/iam_sagemaker_exec_role.png)


For SageMaker managed environments such as SageMaker Studio or SageMaker Notebooks you get use method `get_execution_role()` to retrieve execution role.

In [ ]:
import sagemaker, boto3
from sagemaker import get_execution_role

session = sagemaker.Session()
account = boto3.client('sts').get_caller_identity().get('Account')
role = f"arn:aws:iam::{account}:role/service-role/AmazonSageMaker-ExecutionRole-<YOUR_ROLE_ID>" 
# role = get_execution_role() # this method allows to automatically get attached role from SageMaker managed runtimes. Will not work on local environment.

In [ ]:
from sagemaker.pytorch import PyTorch
import os

pytorch_estimator = PyTorch(
                        session=session,
                        entry_point=f'{os.getcwd()}/sources/cifar10.py',
                        role=role,
                        instance_type="ml.m4.xlarge",
                        instance_count=1,
                        job_name="test",
                        framework_version="1.9.0",
                        py_version="py38",
                        hyperparameters={
                            "epochs": 1,
                            "batch-size": 16
                            }
                        )

pytorch_estimator.fit()


Once training job is done, you can explore locally training results and where output artifacts have been stored.

In [ ]:
pytorch_estimator.latest_training_job.describe()